# Kom i gang med DAT801-2021-konkurransen

***UNDER CONSTRUCTION***


Denne notebooken beskriver hvordan du kan:
1. Laste inn data: vi bruker Pandas til å laste inn trenings- og test-data
2. Undersøke features: vi undersøker et par features via statistikk og enkle plots.
3. Submitte dine prediksjoner for scoring (og plassering på leaderboard): etter at du har laget en modell og produsert prediksjoner på testdata så må du samle disse i en fil som Kaggle kan bruke til scoring. 

_Hvis du vil bruke denne notebooken som utgangspunkt for ditt eget arbeid kan du trykke "Fork"_

Du kan eventuelt bruke Google Colab eller din egen maskin. For å bruke Colab, (i) last ned notebooken på din egen maskin, (ii) gå til https://colab.research.google.com, logg inn, og last opp notebooken. 

# Setup

In [1]:
%matplotlib inline
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt
from pathlib import Path

# Last inn data

## Hvis du arbeider lokalt

Last ned data via Data-menyen på konkurransens hjemmeside.

## Hvis du bruker Colab

Kjør følgende celle:

In [2]:
#train = pd.read_csv('https://raw.githubusercontent.com/alu042/DAT801/master/KaggleInClass/train.csv')
#test = pd.read_csv('https://raw.githubusercontent.com/alu042/DAT801/master/KaggleInClass/test.csv')
#sampleSubmission = pd.read_csv('https://raw.githubusercontent.com/alu042/DAT801/master/KaggleInClass/sampleSubmission.csv')

## Hvis du bruker Kaggle

In [3]:
DATA = Path('../input/dat801-2021-assignment1')
list(DATA.iterdir())

[PosixPath('../input/dat801-2021-assignment1/sample_submission.csv'),
 PosixPath('../input/dat801-2021-assignment1/train.csv'),
 PosixPath('../input/dat801-2021-assignment1/test.csv')]

In [4]:
train = pd.read_csv(DATA/'train.csv')
test = pd.read_csv(DATA/'test.csv')
sampleSubmission = pd.read_csv(DATA/'sample_submission.csv')

# Ta en titt på data

In [5]:
train.head()

,id,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,...,f_66,f_67,f_68,f_69,f_70,f_71,f_72,f_73,f_74,target
0,161363,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,8.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,6
1,78028,16.0,0.0,1.0,1.0,6.0,2.0,2.0,2.0,14.0,...,0.0,41.0,3.0,0.0,0.0,0.0,2.0,1.0,1.0,7
2,35324,0.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,5
3,67966,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1
4,110279,3.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,NaN,...,0.0,0.0,1.0,5.0,4.0,0.0,0.0,0.0,1.0,5


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 77 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      150000 non-null  int64  
 1   f_0     147000 non-null  float64
 2   f_1     148500 non-null  float64
 3   f_2     149250 non-null  float64
 4   f_3     149250 non-null  float64
 5   f_4     148500 non-null  float64
 6   f_5     149250 non-null  float64
 7   f_6     147000 non-null  float64
 8   f_7     147000 non-null  float64
 9   f_8     149250 non-null  float64
 10  f_9     148500 non-null  float64
 11  f_10    148500 non-null  float64
 12  f_11    148500 non-null  float64
 13  f_12    148500 non-null  float64
 14  f_13    148500 non-null  float64
 15  f_14    149250 non-null  float64
 16  f_15    147000 non-null  float64
 17  f_16    149250 non-null  float64
 18  f_17    148500 non-null  float64
 19  f_18    148500 non-null  float64
 20  f_19    148500 non-null  float64
 21  f_20    14

Vi ser at det er 150.000 instanser i treningsdata. Det mangler noen verdier i hver av features (som er noe en må takle ved hjelp av imputering eller lignende).

In [7]:
test.head()

,id,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,...,f_65,f_66,f_67,f_68,f_69,f_70,f_71,f_72,f_73,f_74
0,105982,0,0,10,0,0,0,0,0,1,...,0,0,1,0,1,0,0,0,0,0
1,137351,0,0,0,0,0,1,0,0,0,...,0,0,0,0,6,0,0,0,0,0
2,151824,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,152873,1,2,0,1,2,1,1,2,0,...,4,0,1,1,0,1,1,0,0,0
4,61900,3,0,0,0,0,0,0,0,0,...,0,0,3,0,3,0,0,0,0,0


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 76 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      50000 non-null  int64
 1   f_0     50000 non-null  int64
 2   f_1     50000 non-null  int64
 3   f_2     50000 non-null  int64
 4   f_3     50000 non-null  int64
 5   f_4     50000 non-null  int64
 6   f_5     50000 non-null  int64
 7   f_6     50000 non-null  int64
 8   f_7     50000 non-null  int64
 9   f_8     50000 non-null  int64
 10  f_9     50000 non-null  int64
 11  f_10    50000 non-null  int64
 12  f_11    50000 non-null  int64
 13  f_12    50000 non-null  int64
 14  f_13    50000 non-null  int64
 15  f_14    50000 non-null  int64
 16  f_15    50000 non-null  int64
 17  f_16    50000 non-null  int64
 18  f_17    50000 non-null  int64
 19  f_18    50000 non-null  int64
 20  f_19    50000 non-null  int64
 21  f_20    50000 non-null  int64
 22  f_21    50000 non-null  int64
 23  f_22    500

Det er 50.000 test-instanser, og det ser ikke ut til å mangle noen features. 

# Utforsk data, preprosesser data, utforsk modeller ("model selection"), prediker & evaluer

Jeg overlater disse delene til dere. Du bør utforske features via statistikk og plots, forsøke å avdekke sammenhenger mellom features og mellom features og labels ved å undersøke korrelasjoner og lignende, og gjerne forsøke å konstruere nye features ved å kombinere eksisterende. Før du kan trene modeller må du sørge for at data er på en egnet form. Du må blant annet finne ut hvordan du best kan takle manglende verdier ("missing values"). Husk å designe et grundig evalueringsoppsett som du kan bruke underveis i utforskingen av modeller. Kryss-validering er naturlig. Bør du kanskje vurdere stratifisert splitting i trening- og validerings-data? Hva med ensembling av flere modeller?

# Submission

Etter at du har trent din modell / dine modeller og produsert prediksjoner på test-data, så må du lage en CSV-fil på korrekt form som kan scores av Kaggle. Den skal ha samme form som `sample_submission.csv`:

In [9]:
sampleSubmission.head()

,id,target
0,105982,7
1,137351,7
2,151824,7
3,152873,7
4,61900,7


Her er en fremgangsmåte:

Vi trenger først en vektor med tilfeldige prediksjoner:

In [10]:
target = np.random.choice(range(9), size=len(test))
target

array([8, 4, 4, ..., 8, 4, 0])

Vi kan så bygge en dataframe via en Python dictionary med nøkler ("keys") 'id' og 'target':

In [11]:
submission = pd.DataFrame({'id': test['id'], 'target': target})

In [12]:
submission.head()

,id,target
0,105982,8
1,137351,4
2,151824,4
3,152873,2
4,61900,4


Denne kan lagres som en CSV vi kan laste opp på Kaggle for scoring:

In [13]:
submission.to_csv('submission.csv', index=False)